In [18]:
import jax
import numpyro
import blackjax
import numpy as np
import numpyro.distributions as dist
from numpyro.infer.util import initialize_model
from numpyro.infer.reparam import TransformReparam

In [19]:
def eight_schools(J, sigma, y=None):
    mu = numpyro.sample('mu', dist.Normal(2, 5))
    tau = numpyro.sample('tau', dist.HalfCauchy(5))
    with numpyro.plate('J', J):
        theta = numpyro.sample('theta', dist.Normal(mu, tau))
        numpyro.sample('obs', dist.Normal(theta, sigma), obs=y)

In [20]:
# Model initialization values
J = 8
y = np.array([28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0])
sigma = np.array([15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0])

In [21]:
rng_key = jax.random.PRNGKey(0)
init_params, potential_fn_gen, *_ = initialize_model(
    rng_key,
    eight_schools,
    model_args=(J, sigma, y),
    dynamic_args=True,
)

In [22]:
logdensity_fn = lambda position: -potential_fn_gen(J, sigma, y)(position)
initial_position = init_params.z

In [23]:
num_warmup = 150

adapt = blackjax.window_adaptation(
    blackjax.nuts, logdensity_fn
)

In [24]:
(last_state, parameters), intermediate_states = adapt.run(rng_key, initial_position, num_warmup)

In [25]:
parameters

{'step_size': Array(0.2521476, dtype=float32),
 'inverse_mass_matrix': Array([ 5.30525   ,  0.31019863, 10.508795  ,  7.5516353 ,  9.697434  ,
        27.249842  , 10.198741  , 14.505366  , 38.56407   , 13.96518   ],      dtype=float32)}

In [26]:
ibuffer_size = 75
fbuffer_size = 0
wsize = 0
num_warmup = ibuffer_size + fbuffer_size + wsize  

(last_state, parameters), intermediate_states = adapt.run(rng_key, initial_position, num_warmup, initial_buffer_size = ibuffer_size,
    final_buffer_size = fbuffer_size,
    first_window_size = wsize)

In [28]:
ibuffer_size = 0
fbuffer_size = 0
wsize = 25
num_warmup = ibuffer_size + fbuffer_size + wsize  

(last_state, parameters), intermediate_states = adapt.run(rng_key, last_state[0], num_warmup, initial_step_size = parameters['step_size'] ,initial_buffer_size = ibuffer_size,
    final_buffer_size = fbuffer_size,
    first_window_size = wsize)

In [29]:
parameters

{'step_size': Array(1., dtype=float32),
 'inverse_mass_matrix': Array([ 9.046988  ,  0.48291278, 63.356754  , 16.818571  , 10.634117  ,
        37.218197  , 28.103865  , 15.139331  , 73.642914  , 21.785732  ],      dtype=float32)}

In [30]:
ibuffer_size = 0
fbuffer_size = 75
wsize = 0
num_warmup = ibuffer_size + fbuffer_size + wsize  

(last_state, parameters), intermediate_states = adapt.run(rng_key, last_state[0], num_warmup, initial_buffer_size = ibuffer_size,
    final_buffer_size = fbuffer_size,
    first_window_size = wsize)

In [31]:
parameters

{'step_size': Array(0.21120399, dtype=float32),
 'inverse_mass_matrix': Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)}

In [32]:
kernel = blackjax.nuts(logdensity_fn, **parameters).step

In [33]:
def inference_loop(rng_key, kernel, initial_state, num_samples):
    @jax.jit
    def one_step(state, rng_key):
        state, info = kernel(rng_key, state)
        return state, (state, info)

    keys = jax.random.split(rng_key, num_samples)
    extra, (states, infos) = jax.lax.scan(one_step, initial_state, keys)

    return states, (
        infos.acceptance_rate,
        infos.is_divergent,
        infos.num_integration_steps,
    )

In [34]:
num_sample = 5

states, infos = inference_loop(rng_key, kernel, last_state, num_sample)
_ = states.position["mu"].block_until_ready()

In [35]:
states

HMCState(position={'mu': Array([4.1346045, 5.239526 , 5.257296 , 5.8142715, 4.757877 ], dtype=float32), 'tau': Array([ 1.5930321 ,  0.91519094, -0.08110353,  0.84623045,  0.95862246],      dtype=float32), 'theta': Array([[ 7.0328045,  0.3239852,  5.8964787,  2.3316412,  5.940436 ,
        12.440558 ,  5.015493 ,  2.185249 ],
       [ 5.4980054,  8.767157 ,  2.8952746,  5.2418   ,  9.513682 ,
         2.388644 ,  6.772124 ,  1.4895481],
       [ 5.702556 ,  5.3514857,  4.4745126,  4.2809114,  3.8527822,
         3.1236255,  6.121111 ,  5.3896837],
       [ 2.8833528,  2.8261654,  8.288954 ,  5.171654 ,  3.7486272,
         5.43755  ,  6.1528983,  8.864385 ],
       [ 4.9101114,  1.6866566, 10.314992 ,  7.352324 ,  2.8150902,
         5.3730884,  4.4259562,  8.040459 ]], dtype=float32)}, logdensity=Array([-56.719673, -53.676243, -46.485077, -51.895256, -53.775112],      dtype=float32), logdensity_grad={'mu': Array([ 0.24900192, -0.02534748, -4.5544415 , -0.7306458 ,  0.8975458 ],      dt

HMCState(position={'mu': Array([9.922787 , 2.6043172, 3.7035708, 8.089965 , 7.3719726], dtype=float32), 'tau': Array([-2.1415424, -0.4907572,  1.4969707,  0.7235381,  0.4811546],      dtype=float32), 'theta_base': Array([[ 0.011798  , -0.6990618 ,  0.5800221 ,  0.6235273 , -0.0350373 ,
        -0.06943975, -2.394491  , -0.13232218],
       [ 0.23016587, -1.844681  ,  0.89817107, -0.6893593 , -1.3917323 ,
         1.8075554 ,  2.0827086 ,  0.20634526],
       [ 0.95044273,  1.6913677 , -0.5630975 ,  0.8642062 ,  1.5816722 ,
        -1.4738693 ,  1.5239587 , -0.8587075 ],
       [-1.0939564 , -0.8527685 , -1.012984  , -0.47866136, -2.1197004 ,
         1.2409846 ,  0.27299595,  2.669759  ],
       [-0.3839416 , -1.2815466 , -0.20384768,  0.45797092, -2.280699  ,
         1.0681396 , -0.48478994,  1.9311254 ]], dtype=float32)}, logdensity=Array([-49.38864 , -50.07078 , -47.313896, -50.438282, -48.44399 ],      dtype=float32), logdensity_grad={'mu': Array([-0.5289164 ,  0.2025292 , -0.11358353, -0.29387444, -0.22782859],      dtype=float32), 'tau': Array([ 0.972792  ,  1.0703826 , -0.6049909 ,  0.6082729 ,  0.72445697],      dtype=float32), 'theta_base': Array([[-2.3605300e-03,  6.9689953e-01, -5.8598340e-01, -6.2643600e-01,
         1.9202059e-02,  6.0784940e-02,  2.4043100e+00,  1.3308096e-01],
       [-1.6145459e-01,  1.8846242e+00, -9.1288722e-01,  7.1373290e-01,
         1.3709313e+00, -1.8212701e+00, -1.9962667e+00, -1.8883181e-01],
       [-5.5228788e-01, -1.8370658e+00,  4.9000901e-01, -8.8506830e-01,
        -2.2309687e+00,  1.6172142e+00, -1.1894215e+00,  1.0260315e+00],
       [ 1.2970624e+00,  8.8716203e-01,  9.4048995e-01,  4.7690460e-01,
         1.9995674e+00, -1.4053854e+00, -8.0283418e-02, -2.6799037e+00],
       [ 5.3674161e-01,  1.3252552e+00,  1.4038040e-01, -4.7285253e-01,
         2.1871793e+00, -1.1764501e+00,  6.6943574e-01, -1.9236170e+00]],      dtype=float32)})

In [36]:
acceptance_rate = np.mean(infos[0])
num_divergent = np.mean(infos[1])

print(f"\Average acceptance rate: {acceptance_rate:.2f}")
print(f"There were {100*num_divergent:.2f}% divergent transitions")

\Average acceptance rate: 0.88
There were 0.00% divergent transitions
